In [ ]:
#main
import numpy as np
import pandas as pd
import scipy as sp
import os
import sys
import shelve 
import string
import time
import datetime
import re

In [ ]:
# support
import corelib

# csv to pd loader
from corelib.data_prep import loader

# reduce memory usage for pd-index
from corelib.data_prep import reduce_mem_usage
from corelib.data_prep import reduce_obj_mem_usage

# searching for unical ordered values
from corelib.data_prep import search_func

#dump data with shelve
from corelib.data_dump import shelve_dump

### dump from this point

In [ ]:
# data = loader(index_col='family_id', dtype={'assigned_day': np.float64})
data = loader()
data.keys()

In [ ]:
df_train, df_test = data.values()
del data

In [ ]:
# reduce memory usage
df_train = reduce_mem_usage(df_train)
df_test = reduce_mem_usage(df_test)

In [ ]:
data = df_train, df_test

In [ ]:
# Dump loaded and prepared data
shelve_dump(data, path='loaded_data', task='s')

### to this point

In [ ]:
# Dump open (prepared dataset)
df_train, df_test = shelve_dump(path='loaded_data', task='o')